## Final Project Submission

Please fill out:
* Student name: 
* Student pace: self paced / part time / full time
* Scheduled project review date/time: 
* Instructor name: 
* Blog post URL:


In [25]:
#python libraries
import sqlite3
import pandas as pd

Loading the dataset for Box office mojo and Rotten tomatoes

In [26]:
#load dataset for Box Office Mojo
BoxOfficeMojo=pd.read_csv('zippedData/bom.movie_gross.csv.gz')
BoxOfficeMojo.head()

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010


In [27]:
#load dataset for Rotten Tomatoes
RottenTomatoes=pd.read_csv('zippedData/rt.reviews.tsv.gz',sep='\t',compression='gzip',encoding='latin-1')
print(RottenTomatoes.head())


   id                                             review rating   fresh  \
0   3  A distinctly gallows take on contemporary fina...    3/5   fresh   
1   3  It's an allegory in search of a meaning that n...    NaN  rotten   
2   3  ... life lived in a bubble in financial dealin...    NaN   fresh   
3   3  Continuing along a line introduced in last yea...    NaN   fresh   
4   3             ... a perverse twist on neorealism...     NaN   fresh   

           critic  top_critic         publisher               date  
0      PJ Nabarro           0   Patrick Nabarro  November 10, 2018  
1  Annalee Newitz           0           io9.com       May 23, 2018  
2    Sean Axmaker           0  Stream on Demand    January 4, 2018  
3   Daniel Kasman           0              MUBI  November 16, 2017  
4             NaN           0      Cinema Scope   October 12, 2017  


Inspecting the columns

In [28]:
print(BoxOfficeMojo.columns)


Index(['title', 'studio', 'domestic_gross', 'foreign_gross', 'year'], dtype='object')


In [29]:
print(RottenTomatoes.columns)

Index(['id', 'review', 'rating', 'fresh', 'critic', 'top_critic', 'publisher',
       'date'],
      dtype='object')


In [30]:
#check the info of the datasets
print(BoxOfficeMojo.info())
print(RottenTomatoes.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   2037 non-null   object 
 4   year            3387 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 132.4+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54432 entries, 0 to 54431
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          54432 non-null  int64 
 1   review      48869 non-null  object
 2   rating      40915 non-null  object
 3   fresh       54432 non-null  object
 4   critic      51710 non-null  object
 5   top_critic  54432 non-null  int64 
 6   publisher   54123 non-null  object
 7   date        54432 non-null  object
dt

#BoxOfficemojo dataset has 5 columns and RottenTomatoes dataset has 8 columns. Both datasets have some columns with missing values.

Check for missing values

In [31]:
#Box office Mojo
print(BoxOfficeMojo.isnull().sum())
#Rotten Tomatoes
print(RottenTomatoes.isnull().sum())

title                0
studio               5
domestic_gross      28
foreign_gross     1350
year                 0
dtype: int64
id                0
review         5563
rating        13517
fresh             0
critic         2722
top_critic        0
publisher       309
date              0
dtype: int64


Handle Missing Values

In [52]:
#Remove rows missing important revenue data in Box Office Mojo dataset
BoxOfficeMojo_cleaned=BoxOfficeMojo.dropna(subset=['domestic_gross'])
print(BoxOfficeMojo_cleaned.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3386 entries, 0 to 3386
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   title            3386 non-null   object 
 1   studio           3381 non-null   object 
 2   domestic_gross   3386 non-null   float64
 3   foreign_gross    3386 non-null   float64
 4   year             3386 non-null   int64  
 5   worldwide_gross  3386 non-null   float64
dtypes: float64(3), int64(1), object(2)
memory usage: 185.2+ KB
None


In [53]:
#Fills missing values in Rotten Tomatoes dataset with 'Unknown'
RottenTomatoes_filled=RottenTomatoes.fillna('Unknown')
print(RottenTomatoes_filled.info())


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1135 entries, 0 to 54394
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          1135 non-null   int64 
 1   review      1135 non-null   object
 2   rating      1135 non-null   object
 3   fresh       1135 non-null   object
 4   critic      1135 non-null   object
 5   top_critic  1135 non-null   int64 
 6   publisher   1135 non-null   object
 7   date        1135 non-null   object
dtypes: int64(2), object(6)
memory usage: 79.8+ KB
None


In [54]:
#drop the rows with missing values in Box Office Mojo dataset
BoxOfficeMojo_cleaned = BoxOfficeMojo.dropna()
print(BoxOfficeMojo_cleaned.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3381 entries, 0 to 3386
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   title            3381 non-null   object 
 1   studio           3381 non-null   object 
 2   domestic_gross   3381 non-null   float64
 3   foreign_gross    3381 non-null   float64
 4   year             3381 non-null   int64  
 5   worldwide_gross  3381 non-null   float64
dtypes: float64(3), int64(1), object(2)
memory usage: 184.9+ KB
None


In [55]:
#drop the rows with missing values in Rotten Tomatoes dataset
RottenTomatoes_cleaned = RottenTomatoes.dropna()
print(RottenTomatoes_cleaned.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 598 entries, 0 to 54346
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          598 non-null    int64 
 1   review      598 non-null    object
 2   rating      598 non-null    object
 3   fresh       598 non-null    object
 4   critic      598 non-null    object
 5   top_critic  598 non-null    int64 
 6   publisher   598 non-null    object
 7   date        598 non-null    object
dtypes: int64(2), object(6)
memory usage: 42.0+ KB
None


Remove Duplicate

In [56]:
RottenTomatoes = RottenTomatoes.drop_duplicates(subset='id')
print(RottenTomatoes.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1135 entries, 0 to 54394
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          1135 non-null   int64 
 1   review      1094 non-null   object
 2   rating      672 non-null    object
 3   fresh       1135 non-null   object
 4   critic      1088 non-null   object
 5   top_critic  1135 non-null   int64 
 6   publisher   1124 non-null   object
 7   date        1135 non-null   object
dtypes: int64(2), object(6)
memory usage: 79.8+ KB
None


In [57]:
BoxOfficeMojo = BoxOfficeMojo.drop_duplicates(subset='title')
print(BoxOfficeMojo.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3386 entries, 0 to 3386
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   title            3386 non-null   object 
 1   studio           3381 non-null   object 
 2   domestic_gross   3386 non-null   float64
 3   foreign_gross    3386 non-null   float64
 4   year             3386 non-null   int64  
 5   worldwide_gross  3386 non-null   float64
dtypes: float64(3), int64(1), object(2)
memory usage: 185.2+ KB
None


Convert Money Columns to Numeric(Box Office Mojo)

In [58]:
# Convert gross columns to numeric, coercing errors to NaN
BoxOfficeMojo['domestic_gross'] = pd.to_numeric(BoxOfficeMojo['domestic_gross'], errors='coerce')
BoxOfficeMojo['foreign_gross'] = pd.to_numeric(BoxOfficeMojo['foreign_gross'], errors='coerce')
print(BoxOfficeMojo.info())


<class 'pandas.core.frame.DataFrame'>
Int64Index: 3386 entries, 0 to 3386
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   title            3386 non-null   object 
 1   studio           3381 non-null   object 
 2   domestic_gross   3386 non-null   float64
 3   foreign_gross    3386 non-null   float64
 4   year             3386 non-null   int64  
 5   worldwide_gross  3386 non-null   float64
dtypes: float64(3), int64(1), object(2)
memory usage: 185.2+ KB
None


In [59]:
# Fill NaN values in gross columns before summing, assuming NaN means 0 gross for calculation.
# Adjust this strategy if missing values have a different meaning.
BoxOfficeMojo['domestic_gross'] = BoxOfficeMojo['domestic_gross'].fillna(0)
BoxOfficeMojo['foreign_gross'] = BoxOfficeMojo['foreign_gross'].fillna(0)
BoxOfficeMojo['worldwide_gross'] = BoxOfficeMojo['domestic_gross'] + BoxOfficeMojo['foreign_gross']
print(BoxOfficeMojo[['title', 'domestic_gross', 'foreign_gross', 'worldwide_gross']].head())

                                         title  domestic_gross  foreign_gross  \
0                                  Toy Story 3     415000000.0    652000000.0   
1                   Alice in Wonderland (2010)     334200000.0    691300000.0   
2  Harry Potter and the Deathly Hallows Part 1     296000000.0    664300000.0   
3                                    Inception     292600000.0    535700000.0   
4                          Shrek Forever After     238700000.0    513900000.0   

   worldwide_gross  
0     1.067000e+09  
1     1.025500e+09  
2     9.603000e+08  
3     8.283000e+08  
4     7.526000e+08  


In [39]:
# Display the first few rows with the new column to verify
print(BoxOfficeMojo[['title', 'domestic_gross', 'foreign_gross', 'worldwide_gross']].head())

                                         title  domestic_gross  foreign_gross  \
0                                  Toy Story 3     415000000.0    652000000.0   
1                   Alice in Wonderland (2010)     334200000.0    691300000.0   
2  Harry Potter and the Deathly Hallows Part 1     296000000.0    664300000.0   
3                                    Inception     292600000.0    535700000.0   
4                          Shrek Forever After     238700000.0    513900000.0   

   worldwide_gross  
0     1.067000e+09  
1     1.025500e+09  
2     9.603000e+08  
3     8.283000e+08  
4     7.526000e+08  


#Cleaned Data check

In [50]:
print('Box Office Mojo Cleaned Dataset:')
print(BoxOfficeMojo_cleaned.info())

Box Office Mojo Cleaned Dataset:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2007 entries, 0 to 3353
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           2007 non-null   object 
 1   studio          2007 non-null   object 
 2   domestic_gross  2007 non-null   float64
 3   foreign_gross   2007 non-null   object 
 4   year            2007 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 94.1+ KB
None


In [51]:
print('\nRotten Tomatoes Cleaned Dataset:')
print(RottenTomatoes_cleaned.info())


Rotten Tomatoes Cleaned Dataset:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 33988 entries, 0 to 54424
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          33988 non-null  int64 
 1   review      33988 non-null  object
 2   rating      33988 non-null  object
 3   fresh       33988 non-null  object
 4   critic      33988 non-null  object
 5   top_critic  33988 non-null  int64 
 6   publisher   33988 non-null  object
 7   date        33988 non-null  object
dtypes: int64(2), object(6)
memory usage: 2.3+ MB
None
